In [2]:
!pip install apyori

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [113]:
dataset = pd.read_csv('Groceries_dataset.csv')
dataset.nlargest(n=20, columns = "Member_number")
# dataset.head()

,Member_number,Date,itemDescription
3578,5000,10-02-2015,soda
9340,5000,16-11-2014,bottled beer
11728,5000,09-03-2014,fruit/vegetable juice
19727,5000,10-02-2015,root vegetables
25489,5000,16-11-2014,other vegetables
27877,5000,09-03-2014,onions
34885,5000,10-02-2015,semi-finished bread
837,4999,26-12-2015,bottled water
2625,4999,16-05-2015,butter milk
5427,4999,24-01-2015,tropical fruit


In [147]:
unq_ids = dataset.iloc[:,:2].drop_duplicates().nlargest(n = dataset.shape[0], columns = ['Member_number'])
unq_ids_list = unq_ids.values

In [148]:
transactions = []
for idx in unq_ids_list:
    prods = list(dataset[(dataset.loc[:,'Member_number'] == idx[0]) & (dataset.loc[:,'Date'] == idx[1])].iloc[:,2].values)
    transactions.append(prods)

In [149]:
len(transactions)

14963

In [170]:
transactions[:5]

[['fruit/vegetable juice', 'onions'],
 ['bottled beer', 'other vegetables'],
 ['soda', 'root vegetables', 'semi-finished bread'],
 ['berries', 'onions'],
 ['butter milk', 'whipped/sour cream']]

In [209]:
from apyori import apriori
rules = apriori(transactions = transactions,  min_support=0.0003,min_confidence=0.006, min_lift = 3)


In [210]:
results = list(rules)

In [211]:
results

[RelationRecord(items=frozenset({'specialty chocolate', 'frozen fish'}), support=0.0003341575887188398, ordered_statistics=[OrderedStatistic(items_base=frozenset({'frozen fish'}), items_add=frozenset({'specialty chocolate'}), confidence=0.049019607843137254, lift=3.0689556157190907), OrderedStatistic(items_base=frozenset({'specialty chocolate'}), items_add=frozenset({'frozen fish'}), confidence=0.02092050209205021, lift=3.0689556157190907)]),
 RelationRecord(items=frozenset({'liver loaf', 'fruit/vegetable juice'}), support=0.00040098910646260775, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fruit/vegetable juice'}), items_add=frozenset({'liver loaf'}), confidence=0.011787819253438114, lift=3.52762278978389), OrderedStatistic(items_base=frozenset({'liver loaf'}), items_add=frozenset({'fruit/vegetable juice'}), confidence=0.12, lift=3.5276227897838903)]),
 RelationRecord(items=frozenset({'ham', 'pickled vegetables'}), support=0.0005346521419501437, ordered_statistics=[Orde

In [212]:
def inspect(results):
    lhs = [tuple(res[2][0][0])[0] for res in results]
    rhs = [tuple(res[2][0][1])[0] for res in results]
    supports = [res[1] for res in results]
    confidence = [res[2][0][2] for res in results]
    lifts = [res[2][0][3] for res in results]
    return list(zip(lhs, rhs, supports, confidence, lifts))
resultsdf = pd.DataFrame(inspect(results),columns = ['Left Hand Product', 'Right Hand product', 'Support', 'Confidence', 'Lift'])

In [213]:
resultsdf

,Left Hand Product,Right Hand product,Support,Confidence,Lift
0,frozen fish,specialty chocolate,0.000334,0.049020,3.068956
1,fruit/vegetable juice,liver loaf,0.000401,0.011788,3.527623
2,ham,pickled vegetables,0.000535,0.031250,3.489506
3,meat,roll products,0.000334,0.019841,3.620548
4,misc. beverages,salt,0.000334,0.021186,3.561941
5,misc. beverages,spread cheese,0.000334,0.021186,3.170127
6,seasonal products,soups,0.000334,0.047170,14.704206
7,spread cheese,sugar,0.000401,0.060000,3.387849
8,bottled beer,sausage,0.000334,0.007375,3.805055
9,bottled beer,whole milk,0.000334,0.007375,3.940950


In [214]:
resultsdf.nlargest(n = 10, columns = 'Lift')

,Left Hand Product,Right Hand product,Support,Confidence,Lift
6,seasonal products,soups,0.000334,0.047170,14.704206
10,brown bread,frozen vegetables,0.000334,0.008881,6.644316
16,chewing gum,soda,0.000401,0.033333,5.732950
33,pastry,whole milk,0.000334,0.006460,5.685895
34,rolls/buns,whole milk,0.000334,0.062500,5.599925
21,curd,fruit/vegetable juice,0.000334,0.009921,5.497869
22,curd,margarine,0.000334,0.009921,5.301516
15,canned beer,whole milk,0.000401,0.008547,4.918803
13,butter milk,canned beer,0.000334,0.019011,4.904615
25,ice cream,rolls/buns,0.000334,0.022026,4.453804


In [88]:
from apyori import apriori
rules = apriori(transactions = transactions, min_support = 0.004, min_confidence = 0.2, min_lift = 3)


In [89]:
results = list(rules)
results 

[RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'escalope', 'mushroom cream sauce'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'escalope', 'pasta'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'ground beef', 'herb & pepper'}), support=0.015997866951073192, ordered_statistics=[OrderedStatistic(items_base=frozenset({'herb & pepper'}), items_add=frozenset({'ground beef'}), 

In [90]:
resultsdf = pd.DataFrame(inspect(results),columns = ['Left Hand Product', 'Right Hand product', 'Support', 'Confidence', 'Lift'])
resultsdf.nlargest(n = 20, columns = 'Lift')

,Left Hand Product,Right Hand product,Support,Confidence,Lift
0,"(light cream,)","(chicken,)",0.004533,0.290598,4.843951
2,"(pasta,)","(escalope,)",0.005866,0.372881,4.700812
6,"(pasta,)","(shrimp,)",0.005066,0.322034,4.506672
26,"(frozen vegetables, ground beef)","(spaghetti, mineral water)",0.004399,0.259843,4.350622
9,"(ground beef, eggs)","(herb & pepper,)",0.004133,0.206667,4.178455
5,"(whole wheat pasta,)","(olive oil,)",0.007999,0.271493,4.122410
19,"(herb & pepper, spaghetti)","(ground beef,)",0.006399,0.393443,4.004360
18,"(herb & pepper, mineral water)","(ground beef,)",0.006666,0.390625,3.975683
4,"(tomato sauce,)","(ground beef,)",0.005333,0.377358,3.840659
1,"(mushroom cream sauce,)","(escalope,)",0.005733,0.300699,3.790833
